In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import itertools
import os
import urllib
import tqdm

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import urllib
import requests
from PIL import Image
from io import BytesIO
import logging

import pandas as pd


XPATH = "xpath"
erwo = "*error"
dash = '-'
slash = '/'
logger = logging.getLogger(__name__)

In [ ]:
# Set general settings
url_main = "https://decklist.tistory.com/"
xp_title = '//*[@id="container"]/main/div/div[2]/div[1]/div/div/h2'
xp_man = '//*[@id="container"]/main/div/div[2]/div[2]/div[2]/table/tbody/tr[1]/td[2]'

# Create headless ChromeOptions object
options = Options()
options.add_argument('--headless')

# Create a new ChromeDriverService object with the path to the Chromedriver executable
service = Service('C://chromedriver.exe')

# Initialize Chrome driver with headless options
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
def xp_deck_ind(ind):
    return f'//*[@id="container"]/main/div/div[2]/div[2]/div[2]/p[{ind}]'

In [8]:
def get_deck_info(driver, url_numb):
    url_comb = url_main + str(url_numb)
    driver.get(url_comb)

    # Get title of the Game
    try:
        title_element = driver.find_element(By.XPATH, xp_title).text
        game_cs = title_element.split("(")[0].strip()
        game_date = title_element.split(")")[1].strip()


    except:
        title_element = erwo
        game_cs = erwo
        game_date = erwo
        logger.error(f'{url_numb} | Failed to get | Title info')    

    

    # Get Number of Players
    try:
        man_element = driver.find_element(By.XPATH, xp_man).text
    except:
        man_element = erwo
        logger.error(f'{url_numb} | Failed to get | Man info')  

    # Get Name of the Decks
    deck_element_scrap = []
    i = 1
    while True:
        try:
            element = driver.find_element(By.XPATH, xp_deck_ind(i))
            deck_element_scrap.append(element.text)
            i += 1
        except NoSuchElementException:
            break

    for i in range(len(deck_element_scrap)):
        while '\n' in deck_element_scrap[i]:
            deck_element_scrap[i] = deck_element_scrap[i].replace('\n', '')

    # Modify Name of the Decks
    deck_elements = (filter(lambda x: x != '', deck_element_scrap))
    deck_elements = list(filter(lambda x: x != ' ', deck_elements))
    # # deck_elements = [s.replace('\n', '') for s in deck_elements]
    # for i in range(len(deck_elements)):
    #     while '\n' in deck_elements[i]:
    #         deck_elements[i] = deck_elements[i].replace('\n', '')


    deck_elements_dash = []
    for deck in deck_elements:
        if dash in deck:
            deck = deck.split(dash)[1]
        deck_elements_dash.append(deck)

    deck_elements_slash = []
    for deck in deck_elements_dash:
        if slash in deck:
            deck = deck.split(slash)
            for d in deck:
                deck_elements_slash.append(d)
        else:
            deck_elements_slash.append(deck)

    deck_elements = [s.strip() for s in deck_elements_slash]
    deck_elemetns_text = ' '.join(deck_element_scrap)

    # Get the Images
    img_elements = driver.find_elements(By.TAG_NAME, 'img')
    db_comb = []
    for i, img_element in enumerate(img_elements):
        deck_code = f"{url_numb:04d}-{i+1:03d}"
        filename = f"C:\ML_YGO\Deck_Log\{deck_code}.jpg"
        img_url = img_element.get_attribute('src')

        # Skip GIF images
        if img_url.startswith('data:image/gif'):
            continue  

        # Get the size of the images
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        width, height = img.size

        # Skip small images
        if (width == 343 and height == 353) or width < 50:
            pass
        elif width > 0:
            # Download Image
            urllib.request.urlretrieve(img_url, filename)

            if len(deck_elements) <= i:
                deck_element = erwo
            else:
                deck_element = deck_elements[i]
            
            db_join = [  
                deck_code,
                game_cs,
                game_date,
                man_element, 
                deck_element,
                title_element, 
                deck_elemetns_text
            ]
            db_comb.append(db_join)
    return db_comb

# Run into Function

In [ ]:
#5~594
#600~1003
#1004~1207
#1317~1334
#1334~1347
#600~812 812~1003

In [12]:
master_comb =[]
for i, url_numb in enumerate(range(800,812)):
    print(f'{url_numb} | Running')
    try: 
        db_comb = get_deck_info(driver,url_numb)
    except:
        print(f'{url_numb} | Unexpected Error')
        break
    master_comb.append(db_comb)
    if i% 5 == 0:
        time.sleep(2)
        print("-- Waiting to cool down --")

master_comb_clean = []
for first in master_comb:
    for second in first:
        joined_str = "\t".join(second)
        master_comb_clean.append (joined_str)

dir_write = 'C:\ML_YGO\Output\deck_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_comb_clean:
        fp.write("%s\n" % item)
os.startfile(dir_write)


800 | Failed to get | Title info
800 | Failed to get | Man info


800 | Running
-- Waiting to cool down --
801 | Running
802 | Running
803 | Running
804 | Running
805 | Running
-- Waiting to cool down --
806 | Running
807 | Running
808 | Running
809 | Running
810 | Running
-- Waiting to cool down --
811 | Running


In [ ]:
url_numb = 400
url_comb = url_main + str(url_numb)
driver.get(url_comb)

# Get Name of the Decks
deck_element_scrap = []
i = 1
while True:
    try:
        element = driver.find_element(By.XPATH, xp_deck_ind(i))
        deck_element_scrap.append(element.text)
        i += 1
    except NoSuchElementException:
        break



# Modify Name of the Decks
deck_elements = (filter(lambda x: x != '', deck_element_scrap))
deck_elements = list(filter(lambda x: x != ' ', deck_elements))

# deck_elements = [s.replace('\n', '') for s in deck_elements]

for i in range(len(deck_elements)):
    while '\n' in deck_elements[i]:
        deck_elements[i] = deck_elements[i].replace('\n', '')



In [ ]:
print(deck_element_scrap)

print(deck_elements)

Legacy Code Save Images

In [ ]:
title_element = driver.find_element(By.XPATH, xp_title)
man_element = driver.find_element(By.XPATH, xp_man)
img_elements = driver.find_elements(By.TAG_NAME, 'img')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import urllib

XPATH = "xpath"
TAG_NAME = "tag_name"

# Create headless ChromeOptions object
options = Options()
options.add_argument('--headless')

# Initialize Chrome driver with headless options
driver = webdriver.Chrome('C://cromedriver.exe', options=options)

url_main = "https://decklist.tistory.com/"

for i in range(1010, 1208):
    url_numb = i
    url_comb = url_main + str(url_numb)
    driver.get(url_comb)

    img_elements = driver.find_elements(By.TAG_NAME, 'img')

    for i in range(len(img_elements)):
        deck_code = f"{url_numb:04d}-{i+1:03d}"
        filename = f"C:\ML_YGO\Deck_Log\{deck_code}.jpg"
        img_element = img_elements[i]
        img_url = img_element.get_attribute('src')
        urllib.request.urlretrieve(img_url, filename)
    
    time.sleep(0.5)

# Close the driver
driver.quit()


KONAMI DB

In [ ]:
numb = 7978

In [ ]:
link_f = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link_b = '&request_locale=en'
link = link_f + str(numb) + link_b
driver.get(link)

In [ ]:
xpath_name = '//*[@id="cardname"]/h1'
xpath_pull = '//*[@id="update_list"]'

In [ ]:
dt_name_all = driver.find_element(By.XPATH, xpath_name)
dt_name_all=dt_name_all.text
dt_name_all.split("\n")
print(dt_name_all)

In [ ]:
dt_pull = driver.find_element(By.XPATH, xpath_pull)
dt_pull = dt_pull.text

# dt_pull_j = "".join(dt_pull)
dt_pull_s = dt_pull.split("\n")
dt_pull_s.pop(0)
print(len(dt_pull_s))
print(dt_pull)

In [ ]:
p1 = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
p2 = re.compile(r'^[A-Z0-9]+(?:-[A-Z0-9]+)*$')

rare = [
    'CR',
    'SE', 'EXSE', 'PSE',
    'GUR', 'GSE', 'GR',
    'HR','GH',
    'UR (PR)','PGR',
    'KC', 'KC+R','KC+UR',
    'M', 'M+GR', 'M+SE', 'M+SR',
    'N','R','SR',
    'UR','UL',
    'P','P+ES','P+EXSE','P+HR','P+R','PS','P+SE','P+SR','P+UR',
    '20th SE', 'QCSE',
    '10000 SE',
    'SH','H','STAR','UR (Hobby)','ST', 'MR', 'PL',
]


In [ ]:
check_year=[]
for i in range(len(dt_pull_s)):
    dt_temp = dt_pull_s[i]
    gen = re.search(p1,dt_temp)
    if gen == None:
        pass
    else :
        check_year.append(i)

In [ ]:
check_pack=[]
for i in range(len(dt_pull_s)):
    dt_temp = dt_pull_s[i]
    gen = re.search(p2,dt_temp)
    if gen == None:
        pass
    else :
        check_pack.append(i)

In [ ]:
db_capsule = []
for year in check_year:
    a = dt_pull_s[year]
    try:
        b = dt_pull_s[year+1]
    except:
        b = None
    try:
        c = dt_pull_s[year+2]
    except:
        c = None
    try:
        d = dt_pull_s[year+3]
    except:
        d = None
    db_capsule.append([ a,b,c,d ])

In [ ]:
df = pd.DataFrame (db_capsule, columns = ["Year","Code","Pack","Rarity"])
df

In [ ]:
db_year, db_pack, db_rare = [], [], []

for item in check_year:
    da = dt_pull_s[item]
    try:
        db = dt_pull_s[item+1]
    except:
        db = None
    try:
        dc = dt_pull_s[item+2]
    except:
        dc = None
    try:
        dd = dt_pull_s[item+3]
    except:
        dd = None
    # Append Year
    db_year.append(da)
    # Append Pack
    if db == None:
        db = erwo
    elif re.search(p2,db):
        pass
    else:
        db = erwo
    db_pack.append(db)
    # Append Rarity
    if dd == None :
        k = "N"
    elif re.search(p1,dd):
        k = "N"
    elif  len(dd) > len(dc):
        if re.search(p1,dc):
            k = "N"
        else:
            k = dc
    else :
        if len(dd) > 8:
            k = "N"
        else:
            if re.search(p1,dc):
                k = "N"
            else:
                k = dd
    db_rare.append(k)

    print(f"Year : {str(da)} | Pack : {db} | Rarity : {str(k)}")

In [ ]:
db_year, db_pack, db_rare = [], [], []

for i, item in enumerate(tt):
    try:
        da, db, dc, dd = dt_pull_s[item:item+4]
    except ValueError:
        da = db = dc = dd = None

    db_year.append(da)

    db = db if db and re.search(p2, db) else erwo
    db_pack.append(db)

    if not dd:
        k = "N"
    elif re.search(p1, dd):
        k = "N"
    elif len(dd) > 6 or (len(dc) > len(dd) and re.search(p1, dc)):
        k = dc or "N"
    else:
        k = dd
    db_rare.append(k)

    print(f"Year : {str(da)} | Pack : {db} | Rarity : {str(k)}")

In [ ]:
k =[]

numb2 = []
numb2.append(str(numb))

k.append(numb2)
k.append(db_pack)
k.append(db_rare)

print(k)

list2d = k
merged = list(itertools.chain.from_iterable(list2d))

print(merged)

kk = "\t".join(merged)

print(kk)

카드 수록팩

In [ ]:
filename = 'C:/ML_YGO/Data/konami_code.txt'
with open(filename) as file:
    lines = [line.rstrip() for line in file]
len(lines)

In [ ]:
xpath_name = '//*[@id="cardname"]/h1'
# xpath_name_en = '//*[@id="cardname"]/h1/span'

xpath_pull = '//*[@id="update_list"]'

xpath_type = '//*[@id="CardTextSet"]/div[1]/div[1]/div[1]/span[2]'
xpath_level = '//*[@id="CardTextSet"]/div[1]/div[1]/div[2]/span[2]'
xpath_atk = '//*[@id="CardTextSet"]/div[1]/div[2]/div[1]/span[2]'
xpath_def = '//*[@id="CardTextSet"]/div[1]/div[2]/div[2]/span[2]'
xpath_spc = '//*[@id="CardTextSet"]/div[1]/div[3]/div/p'
xpath_year = '//*[@id="update_list"]/div[2]/div/div/div[1]'

p1 = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
p2 = re.compile(r'^[A-Z0-9]+(?:-[A-Z0-9]+)*$')

In [ ]:
a= 'JUMP-EN036'

In [ ]:
link_f = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link_b = '&request_locale=en'
erwo = "*Blank*"

nline1 = []
nline2 = []
nline3 = []
nline4 = []

for i in tqdm(range(0,12200)):
    numb = lines[i]
    link = link_f+str(numb) + link_b
    driver.get(link)

    try:
        dt_name = driver.find_element(By.XPATH, xpath_name)
        dt_name = dt_name.text
        if "\n" in dt_name:
            dt_name_sp = dt_name.split("\n")
            dt_name_A = dt_name_sp[0]
            dt_name_B = dt_name_sp[1]

    except:
        dt_name_A = erwo
        dt_name_B = erwo

    dt_name_all=[]
    dt_name_all.append(dt_name)
    # dt_name_all.append(dt_name_A)
    # dt_name_all.append(dt_name_B)


    try:
        dt_type = driver.find_element(By.XPATH, xpath_type).text
    except:
        dt_type = erwo

    try:
        dt_level = driver.find_element(By.XPATH, xpath_level).text
    except:
        dt_level = erwo
    
    try:
        dt_atk = driver.find_element(By.XPATH, xpath_atk).text
    except:
        dt_atk = erwo

    try:
        dt_def = driver.find_element(By.XPATH, xpath_def).text
    except:
        dt_def = erwo

    try:
        dt_spc = driver.find_element(By.XPATH, xpath_spc).text
    except:
        dt_spc = erwo

    db_stats = [dt_type, dt_level, dt_atk, dt_def, dt_spc]

    try:
        dt_pull = driver.find_element(By.XPATH, xpath_pull)
        dt_pull = dt_pull.text
    except:
        dt_pull = erwo

    dt_pull_s = dt_pull.split("\n")
    dt_pull_s.pop(0)

    check_year = [m for m, dt_temp in enumerate(dt_pull_s) if re.search(p1, dt_temp)]

    db_year, db_pack, db_rare = [], [], []

    # fill Normal to blank
    for year in check_year:
        da = dt_pull_s[year] #1st
        try:
            db = dt_pull_s[year+1] #2nd
        except IndexError:
            db = None
        try:
            dc = dt_pull_s[year+2] #3rd
        except IndexError:
            dc = None
        try:
            dd = dt_pull_s[year+3] #4th
        except IndexError:
            dd = None

        # Append Year
        db_year.append(da)

        # Append Pack
        if re.search(p2,db):
            pass
        else:
            db = erwo
        db_pack.append(db)

        # Append Rarity
        if dd == None :
            k = "N"
        elif re.search(p1,dd):
            k = "N"
        
        elif  len(dd) > len(dc): # Override 4th to 3rd
            if re.search(p1,dc):
                k = "N"
            else:
                k = dc
        
        else :
            if len(dd) > 10: # For jp 8
                k = "N"
            else:
                if re.search(p1,dc):
                    k = "N"
                else:
                    k = dd
        db_rare.append(k)

    k1 = []
    k2 = []
    k3 = []
    k4 = []
    numb2 = []
    numb2.append(str(numb))
    
    k1.append(numb2)
    k1.append(dt_name_all)
    k1.append(db_pack)

    k2.append(numb2)
    k2.append(dt_name_all)
    k2.append(db_rare)

    k3.append(numb2)
    k3.append(dt_name_all)
    k3.append(db_stats)

    k4.append(numb2)
    k4.append(dt_name_all)
    k4.append(db_year)

    merged1 = list(itertools.chain.from_iterable(k1))
    merged2 = list(itertools.chain.from_iterable(k2))
    merged3 = list(itertools.chain.from_iterable(k3))
    merged4 = list(itertools.chain.from_iterable(k4))

    data1 = "\t".join(merged1)
    data2 = "\t".join(merged2)
    data3 = "\t".join(merged3)
    data4 = "\t".join(merged4)

    # numb2 = str(numb)
    # data1 = '\t'.join([numb2, dt_name_all, db_pack])
    # data2 = '\t'.join([numb2, dt_name_all, db_rare])
    # data3 = '\t'.join([numb2, dt_name_all] + db_stats)
    # data4 = '\t'.join([numb2, dt_name_all] + db_year)
    
    # append the data
    nline1.append(data1)
    nline2.append(data2)
    nline3.append(data3)
    nline4.append(data4)


master = 'C:/ML_YGO/Output/'
f1 = master + 'out_pa.txt'
f2 = master + 'out_ra.txt'
f3 = master + 'out_st.txt'
f4 = master + 'out_yr.txt'

with open(f1, 'w', encoding="utf-8") as fp:
    for item in nline1:
        fp.write("%s\n" % item)

with open(f2, 'w', encoding="utf-8") as fp:
    for item in nline2:
        fp.write("%s\n" % item)

with open(f3, 'w', encoding="utf-8") as fp:
    for item in nline3:
        fp.write("%s\n" % item)

with open(f4, 'w', encoding="utf-8") as fp:
    for item in nline4:
        fp.write("%s\n" % item)

os.startfile(f1)
os.startfile(f2)
os.startfile(f3)
os.startfile(f4)


In [ ]:
import numpy as np

In [ ]:
dir_full_db = 'C:/ML_YGO/Data/rares.txt'
df = pd.read_csv(dir_full_db, sep='\t')
# db_digit = np.asarray(df["Digit"])
# db_name_ko = np.asarray(df["Name_KO"])
# def fill_na_with_code(df):
#     for col in df.columns:
#         if col != 'Code' and df[col].isnull().any():
#             df[col].fillna(df['Code'], inplace=True)
#     return df
# fill_na_with_code(df)

In [ ]:
unique_values = pd.unique(df.values.ravel())
unique_values

카드명 추출 

In [ ]:
filename = 'data.txt'

In [ ]:
with open(filename) as file:
    lines = [line.rstrip() for line in file]

In [ ]:
len(lines)

English Name Scrap

In [ ]:
link1 = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link3 = '&request_locale=ko'

In [ ]:
nline = []

xpath_name_en = '//*[@id="cardname"]/h1/span'

for i in range(0,54):
    link2=lines[i]
    link4 = link1+link2+link3
    driver.get(link4)

    try:
        id = driver.find_element(By.XPATH, xpath_name_en)
        idt = id.text
        k = str(link2)
        kk = k + " - " + idt
        nline.append(kk)

    except:
        print(i)
        print(link2)


In [ ]:
with open('readme.txt', 'w', encoding="utf-8") as fp:
    for item in nline:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Korean Name Scrap

In [ ]:
link_f = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link_b = '&request_locale=ko'
link = link_f + str(numb) + link_b
driver.get(link)

In [ ]:
xpath_name_ja = '//*[@id="cardname"]/h1'

In [ ]:
dt_name = driver.find_element(By.XPATH, xpath_name_ja)
dt_name=dt_name.text
print(dt_name)

In [ ]:
xpath_pack = '//*[@id="update_list"]'

In [ ]:
dt_pack = driver.find_element(By.XPATH, xpath_pack)
dt_pack = dt_pack.text
print(dt_pack)

Japanese Name Scrap

In [ ]:
link_f = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link_b = '&request_locale=ja'
link = link_f + str(numb) + link_b
driver.get(link)

In [ ]:
xpath_name_ja = '//*[@id="cardname"]/h1'

In [ ]:
dt_name = driver.find_element(By.XPATH, xpath_name_ja)
dt_name=dt_name.text
print(dt_name)

In [ ]:
filename = 'data.txt'
with open(filename) as file:
    lines = [line.rstrip() for line in file]
len(lines)

In [ ]:
nline = []

for i in range(0,50):
    link2=lines[i]
    link4 = link1+link2+link3
    driver.get(link4)

    try:
        id = driver.find_element(By.XPATH, xpath_name_ja)
        idt = id.text
        m = idt.split("\n")
        m = "\t".join(m)
        k = str(link2)
        kk = k + "\t" + m
        print(kk)
        nline.append(kk)

    except:
        print(i)
        print(link2)


In [ ]:
with open('readme.txt', 'w', encoding="utf-8") as fp:
    for item in nline:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

BS4

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
home = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid=12950&request_locale=ja'
req = requests.get(home)
req.text
soup = BeautifulSoup(req.content, 'html.parser')

In [ ]:
b = soup.find("div", {"id": "cardname"}).text

In [ ]:
b = soup.find("div", {"id": "update_list"}).text
b

In [ ]:
filename = 'data.txt'
with open(filename) as file:
    lines = [line.rstrip() for line in file]
len(lines)

In [ ]:
link1 = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='
link3 = '&request_locale=ja'

In [ ]:
nline = []

for i in range(0,100):
    link2=lines[i]
    link4 = link1+link2+link3

    req = requests.get(link4)
    req.text
    soup = BeautifulSoup(req.content, 'html.parser')

    try:
        b = soup.find("div", {"id": "cardname"}).text
        b1= b.replace("\n","")
        b2= b1.replace("\t\t","\t")
        b3= b2.replace("\t\t","\t")
        b4 = b3.split("\t")
        b5 = "\t".join(b4)
        k = str(link2)
        kk = k + "\t" + b5
        # print(kk)
        nline.append(kk)

    except:
        print(i)
        print(link2)


In [ ]:
with open('readme_jp.txt', 'w', encoding="utf-8") as fp:
    for item in nline:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')